In [383]:
%%javascript
$('#run_all_cells_below').click()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [384]:
from hypothesis import settings

settings.register_profile("presentation", settings(
    database_file=None, 
    max_examples=100, 
    stateful_step_count=1000,
))

settings.load_profile("presentation")

# Property Based Testing
## Using Hypothesis<br><br><br>
### Amsterdam Python Meetup
### 26 April 2017<br><br><br>
### Daniel Bradburn

Property based testing

Choosing properties

Generating data

Stateful testing

## Property Based Testing

In [385]:
def encode(text):
    """
    Run length encode text - repeated characters are replaced
    by a single instance followed by the count.
    """
    prev, count, result = '', '', []
    
    for curr in text:
        if prev == curr:
            count += 1
        else:
            result.append(f'{prev}{count}')
            prev, count = curr, 1
    else:
        result.append(f'{curr}{count}')
        
    return ''.join(result)

In [386]:
encode('hhellllo')

'h2e1l4o1'

In [387]:
from re import sub

def decode(text):
    """
    Run length decode text - characters followed by a count n
    are replaced by the character repeated n times.
    """
    grouper = lambda m: m.group(1) * int(m.group(2))
    return sub(r'(\D)(\d+)', grouper, text)

In [388]:
decode('h1e1l4o1')

'hellllo'

In [389]:
def test_run_length_encode():
    assert encode('hello') == 'h1e1l2o1'

In [390]:
!sh pytest.sh test_run_length_encode

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 0.60 seconds


In [391]:
def test_run_length_decode():
    assert decode('h1e1l2o1') == 'hello'

In [392]:
!sh pytest.sh test_run_length_decode

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 0.63 seconds


In [393]:
import pytest

examples = ['hello', 'python', 'meetup', 'amsterdam']

@pytest.mark.parametrize('text', examples)
def test_parameterized_run_length_encode_decode(text):
    assert decode(encode(text)) == text

In [394]:
!sh pytest.sh test_parameterized_run_length_encode_decode

....

==================== 29 tests deselected =====================

4 passed, 29 deselected in 0.59 seconds


In [395]:
from random import seed, choice, randint

seed(0)

randletter = lambda _: chr(choice(range(32, 255)))
randrange  = lambda length: range(randint(0, length))
randchars  = lambda max_len: map(randletter, randrange(max_len))
randword   = lambda max_len: ''.join(randchars(max_len))
randwords  = lambda n, max_len: (randword(max_len) for _ in range(n))

@pytest.mark.parametrize('text', randwords(n=5, max_len=5))
def test_fuzzed_run_length_encode_decode(text):
    assert decode(encode(text)) == text

In [396]:
!sh pytest.sh test_fuzzed_run_length_encode_decode

.....

==================== 28 tests deselected =====================

5 passed, 28 deselected in 0.60 seconds


In [397]:
from hypothesis import strategies as st, given

@given(st.text())
def test_property_based_run_length_encode_decode(text):
    assert decode(encode(text)) == text

In [398]:
!sh pytest.sh test_property_based_run_length_encode_decode

F

========================== FAILURES ==========================


________ test_property_based_run_length_encode_decode ________


test_run.py:77: in test_property_based_run_length_encode_decode

test_run.py:78: in test_property_based_run_length_encode_decode

test_run.py:26: in encode

E   UnboundLocalError: local variable 'curr' referenced before assignment

------------------------- Hypothesis -------------------------

Falsifying example: test_property_based_run_length_encode_decode(text='')

==================== 32 tests deselected =====================

1 failed, 32 deselected in 0.74 seconds


In [399]:
def encode_fixed(text):
    """
    Run length encode text - repeated characters are replaced
    by a single instance followed by the count.
    """
    curr, prev, count, result = '', '', '', []
    
    for curr in text:
        if prev == curr:
            count += 1
        else:
            result.append(f'{prev}{count}')
            prev, count = curr, 1
    else:
        result.append(f'{curr}{count}')
        
    return ''.join(result)

In [400]:
@given(st.text())
def test_property_based_fixed_run_length_encode_decode(text):
    assert decode(encode_fixed(text)) == text

In [401]:
!sh pytest.sh test_property_based_fixed_run_length_encode_decode

F

========================== FAILURES ==========================


_____ test_property_based_fixed_run_length_encode_decode _____


test_run.py:101: in test_property_based_fixed_run_length_encode_decode

test_run.py:102: in test_property_based_fixed_run_length_encode_decode

E   AssertionError: assert '01' == '0'



------------------------- Hypothesis -------------------------

Falsifying example: test_property_based_fixed_run_length_encode_decode(text='0')

==================== 32 tests deselected =====================

1 failed, 32 deselected in 0.75 seconds


In [402]:
from hypothesis import settings, Verbosity

@settings(verbosity=Verbosity.verbose)
@given(st.text())
def test_property_based_show_encode_decode(text):
    assert decode(encode_fixed(text)) == text

In [403]:
!sh pytest.sh test_property_based_show_encode_decode

F

========================== FAILURES ==========================


___________ test_property_based_show_encode_decode ___________


test_run.py:108: in test_property_based_show_encode_decode

test_run.py:110: in test_property_based_show_encode_decode

E   AssertionError: assert '01' == '0'



------------------------- Hypothesis -------------------------

Trying example: test_property_based_show_encode_decode(text='\U0007779bS쟫d⍞띍\U0005e51bⅥè\U000dba55[진,J\U00108ff9谟\U000c8cd9\U000aea93Ꚉ𡸶\x1e\x8f\x0bKô')
Trying example: test_property_based_show_encode_decode(text='\U00052520S鍻諗⍜\U0001bf4d\U00051829\U000ef94c𑊗')
Trying example: test_property_based_show_encode_decode(text='\U00052620𐡃º-\U000c31ca')
Trying example: test_property_based_show_encode_decode(text='Ŷ')
Trying example: test_property_based_show_encode_decode(text='¦\U000e104f\U0005c66a\ua7ca\x86\U000d3863鲦Ǒ\x01霺英\n')
Trying example: test_property_based_show_encode_decode(text='\U000108a6\U000e114fǚ')
Trying example: test_propert

In [404]:
from random import seed

seed(0)

@pytest.mark.parametrize('text', randwords(n=5, max_len=15))
def test_fuzzed_more_run_length_encode_decode(text):
    assert decode(encode_fixed(text)) == text

In [405]:
!sh pytest.sh test_fuzzed_more_run_length_encode_decode

.FFF.

========================== FAILURES ==========================


_____ test_fuzzed_more_run_length_encode_decode[o9\xda2] _____


test_run.py:119: in test_fuzzed_more_run_length_encode_decode

E   AssertionError: assert 'oooooooooooo...ÚÚÚÚÚÚÚÚÚÚÚÚÚ' == 'o9Ú2'


___ test_fuzzed_more_run_length_encode_decode[\x98\xaf9z\x8fp\xbc\xc3T\xad] ____

E   AssertionError: assert '\x98¯¯¯¯¯¯¯¯...z\x8fp¼ÃT\xad' == '\x98¯9z\x8fp¼ÃT\xad'


 test_fuzzed_more_run_length_encode_decode[\x91\xfd\xa5b/\xee\xac#7\xd8\xf7\x86\xd5\xf3\xe8] 

E   AssertionError: assert '\x91ý¥b/î¬##...####Ø÷\x86Õóè' == '\x91ý¥b/î¬#7Ø÷\x86Õóè'



==================== 28 tests deselected =====================

3 failed, 2 passed, 28 deselected in 0.70 seconds


In [406]:
from string import digits

@given(st.text(st.characters(blacklist_characters=digits)))
def test_property_based_no_digits_run_length_encode_decode(text):
    assert decode(encode_fixed(text)) == text

In [407]:
!sh pytest.sh test_property_based_no_digits_run_length_encode_decode

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 0.67 seconds


&nbsp;

* Example Based Testing

*  Parameterized Tests

*  Property Based Tests

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• Data generation and shrinking

## Property patterns

In [408]:
@given(st.text())
def test_there_and_back_again(text):
    assert text.encode('utf-8').decode('utf-8') == text

In [409]:
!sh pytest.sh test_there_and_back_again

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 0.67 seconds


In [410]:
@given(st.lists(st.integers(), min_size=1))
def test_round_and_around(c):
    assert c[::-1][::-1] == c

In [411]:
!sh pytest.sh test_round_and_around

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 0.66 seconds


In [412]:
@given(st.integers(), st.integers())
def test_different_paths_same_destination(x, y):
    assert x + y == y + x

In [413]:
!sh pytest.sh test_different_paths_same_destination

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 0.67 seconds


In [414]:
from heapq import heapify, heappop

@given(st.lists(st.integers(), min_size=1))
def test_some_things_never_change(c):
    smallest = min(c)
    heapify(c)
    assert heappop(c) == smallest

In [415]:
!sh pytest.sh test_some_things_never_change

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 0.67 seconds


In [416]:
@given(st.lists(st.integers()))
def test_the_more_things_change_the_more_they_stay_the_same(c):
    assert set(c) == set(set(c))

In [417]:
!sh pytest.sh test_the_more_things_change_the_more_they_stay_the_same

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 0.66 seconds


In [418]:
from dataset import connect

@given(st.lists(st.integers(min_value=0, max_value=1e6), min_size=1))
def test_two_heads_are_better_than_one(numbers):
    
    db = connect('sqlite:///:memory:')
    db['nums'].insert_many({'num': x} for x in numbers)
    
    actual = next(db.query('select sum(num) s from nums'))['s']
    expected = sum(numbers)
   
    assert actual == expected

In [419]:
!sh pytest.sh test_two_heads_are_better_than_one

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 1.17 seconds


&nbsp;

* No errors

* Function pairs

* Commutivity

* Inverse

* Invariants

* Idempotency

* Model Based

## Data generation

In [420]:
from re import findall
from stdnum.isin import is_valid

def extract_isin_codes(text):
    """
    Extract strings conforming to the isin code standard
    from words in text.
    """
    regex = '[a-zA-Z]{2}[a-zA-Z0-9]{9}[0-9]'
    possible_isins = findall(regex, text)
    return filter(is_valid, possible_isins)

In [421]:
from hypothesis import given, strategies as st

@settings(verbosity=Verbosity.verbose)
@given(st.text())
def test_1_extract_isin_codes(text):
    extract_isin_codes(text)

In [422]:
!sh pytest.sh test_1_extract_isin_codes -s

Trying example: test_1_extract_isin_codes(text='\U0007779bS쟫d⍞띍\U0005e51bⅥè\U000dba55[진,J\U00108ff9谟\U000c8cd9\U000aea93Ꚉ𡸶\x1e\x8f\x0bKô')
Trying example: test_1_extract_isin_codes(text='\U00052520S鍻諗⍜\U0001bf4d\U00051829\U000ef94c𑊗')
Trying example: test_1_extract_isin_codes(text='\U00052620𐡃º-\U000c31ca')
Trying example: test_1_extract_isin_codes(text='Ŷ')
Trying example: test_1_extract_isin_codes(text='¦\U000e104f\U0005c66a\ua7ca\x86\U000d3863鲦Ǒ\x01霺英\n')
Trying example: test_1_extract_isin_codes(text='\U000108a6\U000e114fǚ')
Trying example: test_1_extract_isin_codes(text='=\U000e104f(\U0001cc81\x850\x1e\U000504b1')
Trying example: test_1_extract_isin_codes(text='H')
Trying example: test_1_extract_isin_codes(text='3ĺ\x06^\U000875aeÁ¶嵱\n鏿\U0010dbd1+\n:\r\U000d0e91')
Trying example: test_1_extract_isin_codes(text='\U00040833')
Trying example: test_1_extract_isin_codes(text='ꅗ')
Trying example: test_1_extract_isin_codes(text='\U00102444\U00100227\U00080f69\n')
Trying example: test_1_ex

In [423]:
from string import ascii_letters, digits
from stdnum.isin import _country_codes, from_natid

@st.composite
def st_isin_codes(draw):
    country_code = draw(st.sampled_from(_country_codes))
    alphanum = ascii_letters + digits
    natid = draw(st.text(alphanum, min_size=9, max_size=9))
    return from_natid(country_code, natid)

In [424]:
st_isin_codes().example()

'GL47J0IHHZ46'

In [425]:
@given(st.lists(st_isin_codes()), st.lists(st.text()), st.randoms())
def test_2_extract_isin_codes(isins, tokens, random):
    
    tokens += isins
    random.shuffle(tokens)
    text = ' '.join(tokens)
    
    actual = sorted(extract_isin_codes(text))
    expected = sorted(isins)
    
    assert actual == expected 

In [426]:
!sh pytest.sh test_2_extract_isin_codes

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 0.78 seconds


In [427]:
class TextWithIsins:
    
    def __init__(self, isins, tokens, random):
        self.isins, self.tokens = isins, isins + tokens
        random.shuffle(self.tokens)
        
    def __repr__(self):
        return ' '.join(self.tokens)

    
st_text_with_isins = st.builds(TextWithIsins, 
    isins=st.lists(st_isin_codes()),
    tokens=st.lists(st.text()),
    random=st.randoms(),
)

In [428]:
@settings(verbosity=Verbosity.verbose)
@given(st_text_with_isins)
def test_3_extract_isin_codes(text_with_isins):
    actual = sorted(extract_isin_codes(repr(text_with_isins)))
    expected = sorted(text_with_isins.isins)
    assert actual == expected 

In [429]:
!sh pytest.sh test_3_extract_isin_codes -s

Trying example: test_3_extract_isin_codes(text_with_isins=BDUX95ENVQX2 쟫d⍞띍񞔛Ⅵè󛩕[진,J􈿹谟󈳙򮪓Ꚉ𡸶

Kô)
Trying example: test_3_extract_isin_codes(text_with_isins=󰤰󜂊
  CAA9DXJ9CMB6)
Trying example: test_3_extract_isin_codes(text_with_isins=  󣟆 AWI6BAAVAXI9 GWJDHWHHDEE2)
Trying example: test_3_extract_isin_codes(text_with_isins=0 򸶿쎠 񏎍)
Trying example: test_3_extract_isin_codes(text_with_isins=)
Trying example: test_3_extract_isin_codes(text_with_isins=Û=쌶(⦖痕졨񐒱 w>ǂ 
頉"鞙Q ^E  󭐮񕨬񂳠 n
´E􍯑+
:
󐺑 jĝ
򊩱J 󗵒
?蝚¡ ¸ ø󵀩-D󸘥鮇ࠕ AQ0SLBRG2OF9 

  
㙸GN-
T⤝齕[򞚨
 ')
Trying example: test_3_extract_isin_codes(text_with_isins=ڥ𒢥  GSGZRHVLPCB6 REAAQA4ACAC7 )
Trying example: test_3_extract_isin_codes(text_with_isins=GSA8LD9SWAS3 X*"
񹳠n
񤆑êà񰧉䝨v DEHNQXQK1LH8 VNFX3DLTG644 AXP9GDBASEE9 CO3L82ODMJ73 ZMONLA5JR193 XCNBI4CODJF5 AMQ4ZNSZKQG8  EH7CTUCAPXW6 IREGXLOGJQD3 °
  LR8IPOPTL0L3 	I֍` BDPPAYDTLCD8 GMRW0EXSRJV8)
Trying example: test_3_extract_isin_codes(text_with_isins=UAFBA8B7LZF4 FOCHX9W5KMB9 NPVJTBQ0DAS3 󣽕)
Trying ex

In [430]:
nodes = st.floats() | st.booleans() | st.text() | st.none()
children = lambda x: st.lists(x) | st.dictionaries(st.text(), x)
st.recursive(nodes, children).example()

[[{'\n邠·n°': 9007199254740992.0, '䕕\U0009c859': False},
  {'': ['\x05',
    -0.7953042655303273,
    False,
    True,
    '푭렲',
    -inf,
    False,
    5.732881846031734e+18,
    None,
    False,
    None,
    False,
    False],
   '\n': {},
   '@D': ['⻨䃎䟞\nǭť\n믐è萜Ɗ\u1ca9榃Ó', None, -0.99999, inf],
   '\x9c\x8c8\x04(\x07\n崕\U0004f80c\U00078c8a\U000fcfcd': ['ð',
    False,
    None,
    False,
    nan,
    None,
    '',
    -9.929792697524115e+17],
   'Ƣ\x0f': 0.6305258879649471,
   '☶Ŗ\n\U000997b1Â\U0006ea1c\\Ő\x11}': [0.6844653687167109,
    False,
    None,
    7.088699320972906e-140,
    None,
    2.0454715940347337e-175,
    None,
    '\U0007ae0f',
    '\U000dc39e刎'],
   '恅': ['䡞-$', 0.0],
   '\U00107a2a$': False},
  True,
  {'': {'': '\U000cd243ÜòQ\U000d7aabb\U0005be46\U000856cc㛱]\x1bኗ5\n됡ꓸ\U00012870q煚𐅜D.鬶ˁ\U00091709\n\U00057475\n\U0010e6682𗚄\x07',
    "9'$ţᦓ鞔\\Ç筋": True,
    ']\U0009f066Ċ<': True,
    '\x8c儋\x82\U000344a6\U0010b9db⏂\n\U000eadc9\U00073a88\U000e8082': False,
    '\

&nbsp;

* Input data often needs to be a particular shape

* PBT Frameworks offer data generation mechanisms

* Hypothesis is very advanced in this respect

## Stateful Testing

In [431]:
class Queue(object):
    """FIFO queue with a maximum size"""

    def __init__(self, max_size):
        self._buffer = [None] * max_size
        self._in, self._out, self.max_size = 0, 0, max_size

    def put(self, item):
        self._buffer[self._in] = item
        self._in = (self._in + 1) % self.max_size

    def get(self):
        result = self._buffer[self._out]
        self._out = (self._out + 1) % self.max_size
        return result

    def __len__(self):
        return (self._in - self._out) % self.max_size

In [432]:
import itertools

In [433]:
operations = 'new', 'put', 'get', 'size'
list(itertools.permutations(operations))

[('new', 'put', 'get', 'size'),
 ('new', 'put', 'size', 'get'),
 ('new', 'get', 'put', 'size'),
 ('new', 'get', 'size', 'put'),
 ('new', 'size', 'put', 'get'),
 ('new', 'size', 'get', 'put'),
 ('put', 'new', 'get', 'size'),
 ('put', 'new', 'size', 'get'),
 ('put', 'get', 'new', 'size'),
 ('put', 'get', 'size', 'new'),
 ('put', 'size', 'new', 'get'),
 ('put', 'size', 'get', 'new'),
 ('get', 'new', 'put', 'size'),
 ('get', 'new', 'size', 'put'),
 ('get', 'put', 'new', 'size'),
 ('get', 'put', 'size', 'new'),
 ('get', 'size', 'new', 'put'),
 ('get', 'size', 'put', 'new'),
 ('size', 'new', 'put', 'get'),
 ('size', 'new', 'get', 'put'),
 ('size', 'put', 'new', 'get'),
 ('size', 'put', 'get', 'new'),
 ('size', 'get', 'new', 'put'),
 ('size', 'get', 'put', 'new')]

In [434]:
from hypothesis.stateful import RuleBasedStateMachine
from hypothesis.stateful import rule, precondition

In [435]:
class QueueStateMachine(RuleBasedStateMachine):
    
    Actual, Model = Queue, list

    is_created = lambda self: hasattr(self, 'model')
    is_not_created = lambda self: not hasattr(self, 'model')
    is_not_empty = lambda self: hasattr(self, 'model') \
                                and self.model
    
    @precondition(is_not_created)
    @rule(max_size=st.integers(min_value=1, max_value=5))
    def new(self, max_size):
        self.actual = self.Actual(max_size) 
        self.model = self.Model()
        self.max_size = max_size

    @precondition(is_created)
    @rule(item=st.integers())
    def put(self, item):
        self.actual.put(item), self.model.append(item)

    @precondition(is_not_empty)
    @rule()
    def get(self):
        actual, model = self.actual.get(), self.model.pop()
        assert actual == model

    @precondition(is_created)
    @rule()
    def length(self):
        actual, model = len(self.actual), len(self.model)
        assert actual == model

In [436]:
test_queue_stateful_1 = QueueStateMachine.TestCase

In [437]:
class QueueStateMachine(QueueStateMachine):
    # this is a total cheat, the order in which the errors would naturally occur is 
    # system under test, model, specification. However the narrative is a little
    # better when presented as model, specification, system under test, so secretly
    # fix the bug with the system under test here
    class Actual(Queue):
        def __init__(self, max_size):
            super().__init__(max_size + 1)
            
test_queue_stateful_1 = QueueStateMachine.TestCase

In [438]:
!sh pytest.sh test_queue_stateful_1

F

========================== FAILURES ==========================


_______________ test_queue_stateful_1.runTest ________________


test_run.py:299: in length

E   AssertionError: assert 0 == 2

------------------------- Hypothesis -------------------------

Step #1: new(max_size=1)
Step #2: put(item=0)
Step #3: put(item=0)
Step #4: length()

==================== 32 tests deselected =====================

1 failed, 32 deselected in 0.78 seconds


In [439]:
class QueueStateMachine2(QueueStateMachine):
        
    is_not_full = lambda self: (hasattr(self, 'model')
                                and len(self.model) < self.max_size)

    @precondition(is_not_full)
    @rule(item=st.integers())
    def put(self, item):
        super().put(item)
        
test_queue_stateful_2 = QueueStateMachine2.TestCase

In [440]:
!sh pytest.sh test_queue_stateful_2

F

========================== FAILURES ==========================


_______________ test_queue_stateful_2.runTest ________________


test_run.py:293: in get

E   AssertionError: assert 0 == 1

------------------------- Hypothesis -------------------------

Step #1: new(max_size=2)
Step #2: put(item=0)
Step #3: put(item=1)
Step #4: get()

==================== 32 tests deselected =====================

1 failed, 32 deselected in 0.85 seconds


In [441]:
class QueueStateMachine2(QueueStateMachine2):
    # undo the cheating fix
    Actual = Queue

In [442]:
class QueueStateMachine3(QueueStateMachine2):
    
    @precondition(QueueStateMachine2.is_not_full)
    @rule(item=st.integers())
    def put(self, item):
        self.actual.put(item), self.model.insert(0, item)
        
test_queue_stateful_3 = QueueStateMachine3.TestCase

In [443]:
!sh pytest.sh test_queue_stateful_3

F

========================== FAILURES ==========================


_______________ test_queue_stateful_3.runTest ________________


test_run.py:299: in length

E   AssertionError: assert 0 == 1

------------------------- Hypothesis -------------------------

Step #1: new(max_size=1)
Step #2: put(item=0)
Step #3: length()

==================== 32 tests deselected =====================

1 failed, 32 deselected in 0.80 seconds


In [444]:
queue = Queue(max_size=1)
queue._in, queue._out

(0, 0)

In [445]:
queue.put(0)
queue._in, queue._out

(0, 0)

In [446]:
def put(self, item):
    self._buffer[self._in] = item
    self._in = (self._in + 1) % self.max_size

In [447]:
class Queue2(Queue):
    def __init__(self, max_size):
        super(Queue2, self).__init__(max_size + 1)

class QueueStateMachine4(QueueStateMachine3):
    Actual = Queue2
    
test_queue_stateful_4 = QueueStateMachine4.TestCase

In [448]:
!sh pytest.sh test_queue_stateful_4

.

==================== 32 tests deselected =====================

1 passed, 32 deselected in 1.61 seconds


&nbsp;

* Automatic testing of stateful systems

* Interpreting Errors

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• Specification

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• Model

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;• System under test

&nbsp;

# Stop Writing Tests...

# ...Start Generating Them!

# Shrinking - Extract signal from noise

# Look for property patterns

# Stateful testing - Auto integration tests

In [449]:
%%javascript
//$('#clear_all_output').click()

<IPython.core.display.Javascript object>

In [451]:
%%HTML
<link href="https://fonts.googleapis.com/css?family=ABeeZee" rel="stylesheet">
<style>body { font-family: 'ABeeZee', serif !important; }</style>